In [93]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [154]:
def get_selected_africa_coutries_list() -> list:
    processed_path = Path(str(Path(os.getcwd()).parent.absolute()) + '/ProcessedHistograms')
    country_list = sorted(processed_path.glob('*'))
    return [str(country).split('\\')[-1].replace('_', ' ') for country in country_list]

def get_ndvi_evi_data() -> [pd.DataFrame, pd.DataFrame]:
    countries_list_nme = get_selected_africa_coutries_list()
    processed_path = Path(str(Path(os.getcwd()).absolute()) + '/Processed_edvi_data')
    country_list = sorted(processed_path.glob('*'))
    ndvi = []
    evi = []
    ndvi_index_list = None
    evi_index_list = None
    for country in country_list:
        data_list = sorted(country.glob('*.csv'))
        country_nme = str(country).split('\\')[-1]
        country_nme = country_nme.replace('_', ' ')
        if country_nme not in countries_list_nme:
            continue
        for i in data_list:
            file_name = str(i).split('\\')[-1].split('_')[0]
            temp_data = pd.read_csv(str(i), index_col = 0)
            temp_data.columns = [country_nme]        

            if file_name == 'NDVI':        
                if ndvi_index_list is None:
                    ndvi_index_list = temp_data.index.tolist()
                temp_data = temp_data.reset_index(drop=True)
                ndvi.append(temp_data)
            else:
                if evi_index_list is None:
                    evi_index_list = temp_data.index.tolist()
                temp_data = temp_data.reset_index(drop=True)
                evi.append(temp_data)

        column_list.append(country_nme)
    ndvi_df = pd.concat(ndvi, axis=1)
    evi_df = pd.concat(evi, axis=1)
    ndvi_df.index = ndvi_index_list
    evi_df.index = evi_index_list
    return ndvi_df, evi_df


In [155]:
ndvi, evi = get_ndvi_evi_data()
ndvi_mean_df, ndvi_max_df, ndvi_min_df, evi_mean_df, evi_max_df, evi_min_df  = ndvi_evi_monthly_to_yearly(ndvi, evi)

In [143]:
def ndvi_evi_monthly_to_yearly(ndvi, evi):
    ndvi = ndvi.reset_index()
    evi = evi.reset_index()

    ndvi['year'] = ndvi['index'].apply(lambda x: x[:4])
    evi['year'] = evi['index'].apply(lambda x: x[:4])

    ndvi_mean_df = ndvi.groupby(['year']).mean()
    ndvi_max_df = ndvi.groupby(['year']).max()
    ndvi_min_df = ndvi.groupby(['year']).min()

    evi_mean_df = evi.groupby(['year']).mean()
    evi_max_df = evi.groupby(['year']).max()
    evi_min_df = evi.groupby(['year']).min()

    # ndvi_mean_df = ndvi_mean_df.drop(columns=['index'])
    ndvi_max_df = ndvi_max_df.drop(columns=['index'])
    ndvi_min_df = ndvi_min_df.drop(columns=['index'])

    evi_max_df = evi_max_df.drop(columns=['index'])
    evi_min_df = evi_min_df.drop(columns=['index'])
    
    ndvi_mean_df = ndvi_mean_df.T
    ndvi_max_df = ndvi_max_df.T
    ndvi_min_df = ndvi_min_df.T
    evi_mean_df = evi_mean_df.T
    evi_max_df = evi_max_df.T
    evi_min_df = evi_min_df.T
    
    return ndvi_mean_df, ndvi_max_df, ndvi_min_df, evi_mean_df, evi_max_df, evi_min_df 

In [144]:
def get_water_data() -> pd.DataFrame:
    countries_list_nme = get_selected_africa_coutries_list()
    processed_path = Path(str(Path(os.getcwd()).parent.absolute()) + '/WaterProcessed')
    country_list = sorted(processed_path.glob('*'))
    data = []
    for i in country_list:
        f = open(str(i))
        water_data = json.load(f)
        country_name =  str(i).split('\\')[-1].split('.')[-2][:-5].replace('_', ' ')
        water_data['country_nme'] = country_name
        if country_name in countries_list_nme:
            data.append(water_data)
    water_df = pd.DataFrame(data=data)
    water_df = water_df.set_index('country_nme')
    return water_df

In [146]:
import json
water = get_water_data()



In [156]:
ndvi_mean_df

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Algeria,0.115883,0.116630,0.115041,0.119968,0.122355,0.119373,0.121695,0.122524,0.120300,0.127629,...,0.124718,0.125241,0.122612,0.121690,0.119588,0.119825,0.124105,0.124144,0.122400,0.119973
Angola,0.517406,0.542920,0.550686,0.538646,0.546161,0.540457,0.555013,0.547515,0.553609,0.557180,...,0.544932,0.541749,0.553205,0.542868,0.547691,0.554145,0.566382,0.547543,0.568927,0.566186
Benin,0.482851,0.455197,0.494241,0.496213,0.487469,0.481909,0.486366,0.486422,0.477729,0.494983,...,0.479694,0.488446,0.494554,0.459748,0.467278,0.481604,0.504418,0.505006,0.493374,0.499359
Botswana,0.353178,0.349737,0.283728,0.254480,0.312185,0.286121,0.361931,0.299197,0.330871,0.339277,...,0.301596,0.284632,0.351738,0.301978,0.298119,0.367189,0.321632,0.301754,0.343084,0.383191
Burkina Faso,0.344341,0.337310,0.328498,0.355643,0.341705,0.339855,0.343003,0.340377,0.339471,0.346862,...,0.345556,0.347761,0.340657,0.338379,0.341121,0.344173,0.349878,0.355075,0.351338,0.357815
Burundi,0.505740,0.571438,0.534889,0.570174,0.542868,0.553106,0.541012,0.572214,0.551338,0.547986,...,0.544140,0.543281,0.552258,0.564596,0.538107,0.539397,0.558940,0.540861,0.570674,0.562849
Cameroon,0.548484,0.532393,0.565759,0.553742,0.550745,0.551613,0.568905,0.561135,0.564654,0.563243,...,0.545115,0.564761,0.573921,0.568045,0.552785,0.570146,0.593204,0.583230,0.586336,0.608649
Chad,0.207175,0.207559,0.206937,0.209477,0.205252,0.210540,0.209632,0.212431,0.208642,0.203428,...,0.215767,0.216094,0.214451,0.207719,0.206535,0.210396,0.216670,0.223433,0.226767,0.224060
Djibouti,0.111055,0.110198,0.111175,0.111114,0.112122,0.111391,0.116761,0.117869,0.104579,0.107730,...,0.098137,0.103470,0.103640,0.100610,0.103381,0.107459,0.103710,0.108253,0.125873,0.124965
Equatorial Guinea,0.469286,0.485738,0.501926,0.506504,0.506514,0.494200,0.517162,0.516986,0.533972,0.510576,...,0.485078,0.504028,0.499702,0.483290,0.507923,0.486819,0.544997,0.550955,0.551084,0.582959


In [147]:
water

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country_nme,,,,,,,,,,,,,,
Algeria,178,212,223,199,213,206,207,235,219,208,218,197,214,218
Angola,122,117,118,115,127,129,127,130,140,129,127,135,131,124
Benin,26,22,21,24,23,24,23,24,26,26,27,29,27,23
Botswana,65,52,155,91,171,93,133,193,155,130,93,125,179,104
Burkina Faso,29,33,30,29,27,35,32,34,32,27,33,34,31,33
Burundi,175,173,175,175,175,175,175,175,175,174,174,175,175,175
Cameroon,179,176,167,173,176,173,177,177,178,176,179,174,184,175
Chad,196,172,178,161,161,149,162,159,151,173,163,176,186,178
Congo,366,353,363,365,371,376,369,381,379,368,367,382,378,378


In [97]:
def get_ndvi_evi_data() -> [pd.DataFrame, pd.DataFrame]:
    countries_list_nme = get_selected_africa_coutries_list()
    processed_path = Path(str(Path(os.getcwd()).absolute()) + '/Processed_edvi_data')
    country_list = sorted(processed_path.glob('*'))
    ndvi = []
    evi = []
    ndvi_index_list = None
    evi_index_list = None
    for country in country_list:
        data_list = sorted(country.glob('*.csv'))
        country_nme = str(country).split('\\')[-1]
        country_nme = country_nme.replace('_', ' ')
        if country_nme not in countries_list_nme:
            continue
        for i in data_list:
            file_name = str(i).split('\\')[-1].split('_')[0]
            temp_data = pd.read_csv(str(i), index_col = 0)
            temp_data.columns = [country_nme]        

            if file_name == 'NDVI':        
                if ndvi_index_list is None:
                    ndvi_index_list = temp_data.index.tolist()
                temp_data = temp_data.reset_index(drop=True)
                ndvi.append(temp_data)
            else:
                if evi_index_list is None:
                    evi_index_list = temp_data.index.tolist()
                temp_data = temp_data.reset_index(drop=True)
                evi.append(temp_data)

        column_list.append(country_nme)
    ndvi_df = pd.concat(ndvi, axis=1)
    evi_df = pd.concat(evi, axis=1)
    ndvi_df.index = ndvi_index_list
    evi_df.index = evi_index_list
    return ndvi_df, evi_df


In [99]:
edvi, evi = get_ndvi_evi_data() 

In [100]:
edvi

,Algeria,Angola,Benin,Botswana,Burkina Faso,Burundi,Cameroon,Chad,Djibouti,Equatorial Guinea,...,Seychelles,Sierra Leone,Somalia,South Sudan,Sudan,Togo,Tunisia,Uganda,Zambia,Zimbabwe
2000-02-29,0.119710,0.573375,0.312306,0.474192,0.235228,0.633466,0.453999,0.154605,0.107117,0.360360,...,0.441021,0.575295,0.194773,0.290795,0.135764,0.313422,0.209701,0.356555,0.609958,0.523655
2000-03-31,0.122246,0.608626,0.322919,0.484635,0.223424,0.520000,0.497652,0.157876,0.104077,0.625863,...,0.505551,0.584207,0.190937,0.308131,0.134932,0.337573,0.203764,0.370592,0.636947,0.627199
2000-04-30,0.120680,0.634246,0.397430,0.450450,0.253756,0.576853,0.595779,0.160841,0.101973,0.648138,...,0.637270,0.672749,0.210000,0.381108,0.135524,0.433347,0.180286,0.488093,0.647880,0.592674
2000-05-31,0.113886,0.568838,0.481520,0.387233,0.290165,0.568648,0.633773,0.171842,0.101129,0.599893,...,0.600569,0.591976,0.308958,0.490857,0.141498,0.519485,0.151341,0.573163,0.588064,0.528775
2000-06-30,0.113110,0.480478,0.598962,0.334328,0.365381,0.495615,0.610593,0.203983,0.089478,0.450221,...,0.517206,0.587868,0.256129,0.584987,0.154811,0.609682,0.143638,0.542977,0.513630,0.478984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,0.114725,0.442430,0.563597,0.264463,0.548335,0.492335,0.591884,0.332245,0.133095,0.538557,...,0.585590,0.553512,0.211508,0.736194,0.264411,0.575510,0.147342,0.575975,0.409694,0.344930
2021-09-30,0.115720,0.496802,0.675826,0.265183,0.537632,0.488765,0.694814,0.299718,0.134542,0.472282,...,0.577420,0.769469,0.216801,0.702812,0.254216,0.686067,0.148515,0.589992,0.416903,0.328419
2021-10-31,0.114992,0.502759,0.658491,0.279810,0.450608,0.491144,0.654200,0.259137,0.134744,0.550963,...,0.592214,0.783501,0.242938,0.654796,0.214621,0.686908,0.156120,0.589640,0.475444,0.344870
2021-11-30,0.118332,0.576605,0.580987,0.312053,0.357627,0.588458,0.659520,0.222884,0.139025,0.656958,...,0.605264,0.761958,0.245384,0.593898,0.182878,0.625906,0.171457,0.588652,0.522475,0.397371


In [90]:
countries_list_nme = get_selected_africa_coutries_list()
# print(countries_list_nme)
processed_path = Path(str(Path(os.getcwd()).absolute()) + '/Processed_edvi_data')
country_list = sorted(processed_path.glob('*'))
# print(country_list)
ndvi = []
evi = []
ndvi_index_list = None
evi_index_list = None

country = country_list[0]
for country in country_list:

    data_list = sorted(country.glob('*.csv'))
    country_nme = str(country).split('\\')[-1]
    country_nme = country_nme.replace('_', ' ')
    
        
    for i in data_list:
        
        file_name = str(i).split('\\')[-1].split('_')[0]
        temp_data = pd.read_csv(str(i), index_col = 0)
        temp_data.columns = [country_nme]        
        
        if file_name == 'NDVI':        
            if ndvi_index_list is None:
                ndvi_index_list = temp_data.index.tolist()
            temp_data = temp_data.reset_index(drop=True)
            ndvi.append(temp_data)
        else:
            if evi_index_list is None:
                evi_index_list = temp_data.index.tolist()
            temp_data = temp_data.reset_index(drop=True)
            evi.append(temp_data)

    column_list.append(country_nme)
ndvi_df = pd.concat(ndvi, axis=1)
evi_df = pd.concat(evi, axis=1)
ndvi_df.index = ndvi_index_list
evi_df.index = evi_index_list


In [89]:
countries_list_nme = get_selected_africa_coutries_list()
# print(countries_list_nme)
processed_path = Path(str(Path(os.getcwd()).absolute()) + '/Processed_edvi_data')
country_list = sorted(processed_path.glob('*'))
# print(country_list)
ndvi = []
evi = []
ndvi_index_list = None
evi_index_list = None

country = country_list[0]
for country in country_list:

    data_list = sorted(country.glob('*.csv'))
    country_nme = str(country).split('\\')[-1]
    country_nme = country_nme.replace('_', ' ')
    
    if country_nme not in ['Madagascar', 'Mozambique', 'Cameroon', 'Nigeria']:
        continue
        
    for i in data_list:
        
        temp_data = pd.read_csv(str(i), index_col = 0)
        temp_data = temp_data.drop_duplicates()   
        temp_data = temp_data.iloc[:-4,:]
        temp_data.to_csv(str(i))
        


In [91]:
ndvi_df

,Algeria,Angola,Benin,Botswana,Burkina Faso,Burundi,Cameroon,Chad,Comoros,Djibouti,...,Seychelles,Sierra Leone,Somalia,South Sudan,Sudan,Togo,Tunisia,Uganda,Zambia,Zimbabwe
2000-02-29,0.119710,0.573375,0.312306,0.474192,0.235228,0.633466,0.453999,0.154605,0.607523,0.107117,...,0.441021,0.575295,0.194773,0.290795,0.135764,0.313422,0.209701,0.356555,0.609958,0.523655
2000-03-31,0.122246,0.608626,0.322919,0.484635,0.223424,0.520000,0.497652,0.157876,0.671418,0.104077,...,0.505551,0.584207,0.190937,0.308131,0.134932,0.337573,0.203764,0.370592,0.636947,0.627199
2000-04-30,0.120680,0.634246,0.397430,0.450450,0.253756,0.576853,0.595779,0.160841,0.738394,0.101973,...,0.637270,0.672749,0.210000,0.381108,0.135524,0.433347,0.180286,0.488093,0.647880,0.592674
2000-05-31,0.113886,0.568838,0.481520,0.387233,0.290165,0.568648,0.633773,0.171842,0.739443,0.101129,...,0.600569,0.591976,0.308958,0.490857,0.141498,0.519485,0.151341,0.573163,0.588064,0.528775
2000-06-30,0.113110,0.480478,0.598962,0.334328,0.365381,0.495615,0.610593,0.203983,0.693895,0.089478,...,0.517206,0.587868,0.256129,0.584987,0.154811,0.609682,0.143638,0.542977,0.513630,0.478984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,0.114725,0.442430,0.563597,0.264463,0.548335,0.492335,0.591884,0.332245,0.668369,0.133095,...,0.585590,0.553512,0.211508,0.736194,0.264411,0.575510,0.147342,0.575975,0.409694,0.344930
2021-09-30,0.115720,0.496802,0.675826,0.265183,0.537632,0.488765,0.694814,0.299718,0.636688,0.134542,...,0.577420,0.769469,0.216801,0.702812,0.254216,0.686067,0.148515,0.589992,0.416903,0.328419
2021-10-31,0.114992,0.502759,0.658491,0.279810,0.450608,0.491144,0.654200,0.259137,0.686969,0.134744,...,0.592214,0.783501,0.242938,0.654796,0.214621,0.686908,0.156120,0.589640,0.475444,0.344870
2021-11-30,0.118332,0.576605,0.580987,0.312053,0.357627,0.588458,0.659520,0.222884,0.658832,0.139025,...,0.605264,0.761958,0.245384,0.593898,0.182878,0.625906,0.171457,0.588652,0.522475,0.397371


In [34]:
data_list = sorted(country.glob('*.csv'))

In [37]:
i = data_list[0]

In [38]:
pd.read_csv(str(i), index_col =0)

,EVI
2000-02-29,0.084882
2000-03-31,0.087985
2000-04-30,0.088525
2000-05-31,0.084090
2000-06-30,0.083983
...,...
2021-08-31,0.084065
2021-09-30,0.084086
2021-10-31,0.082394
2021-11-30,0.082783
